### 필요한 라이브러리 불러오기

In [17]:
!pip install konlpy

In [18]:
import pandas as pd
import urllib.request
import matplotlib.pyplot as plt
%matplotlib inline
import re
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os
from tensorflow.keras.datasets import reuters
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
from tqdm import tqdm

### 데이터 불러오기

In [19]:
train = pd.read_csv('../input/dacon-kor-opinion-classification/train.csv', encoding = 'utf-8')
test = pd.read_csv('../input/dacon-kor-opinion-classification/test.csv', encoding = 'utf-8')
sample_submission = pd.read_csv('../input/dacon-kor-opinion-classification/sample_submission.csv', encoding = 'utf-8')


### 데이터 전처리하기

In [20]:
train.info() # shape: (40000, 3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   index     40000 non-null  int64 
 1   category  40000 non-null  int64 
 2   data      39992 non-null  object
dtypes: int64(2), object(1)
memory usage: 937.6+ KB


In [21]:
train.head()

,index,category,data
0,0,2,신혼부부위한 주택정책 보다 보육시설 늘려주세요.. 국민세금으로 일부를 위한 정책펴지...
1,1,0,학교이름에 '남자'도 붙여주세요. 울산여자중학교에 재학중인 학생입니다 최근 양성평등...
2,2,1,"빙상연맹, 대한축구협회등 각종 체육협회의 비리를 철저하게 밝혀주세요.. 최근 동계올..."
3,3,1,"티비 12세,15세 관람가도 연령확인 의무화 하자.. 제기 에전에 티비를 보다가 잠..."
4,4,1,무더운 여름철엔 남성들도 시원한 자율복장을 해야. 무더운 여름철에는 남성들도 노넥타...


In [22]:
sum(train['data'].isnull())

8

In [23]:
train = train.dropna(how = 'any')
train['data'] = train['data'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣]", "")
test['data'] = test['data'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣]", "")
# 불용어: 의존 형태소
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','을']

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


In [24]:
okt = Okt()

In [ ]:
X_train = []
for sentence, i in zip(train['data'], tqdm(range(len(train['data'])))):
    temp_X = []
    temp_X = okt.morphs(sentence, stem = True) # Okt 형태소 분석
    temp_X = [word for word in temp_X if not word in stopwords]
    X_train.append(temp_X)

  1%|          | 286/39992 [23:17<55:11:52,  5.00s/it] 

In [ ]:
X_test = []
for sentence in test['data']:
    temp_X = []
    temp_X = okt.morphs(sentence, stem = True)
    temp_X = [word for word in temp_X if not word in stopwords]
    X_test.append(temp_X)

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [ ]:
vocab_size = 30000
tokenizer = Tokenizer(vocab_size)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train) # 토큰화
X_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
max_len = 500
X_train = pad_sequences(X_train, maxlen = max_len)
X_test = pad_sequences(X_test, maxlen = max_len)

In [ ]:
y_train = to_categorical(train['category'])

### 모델 생성 및 훈련하기

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 120))
model.add(LSTM(120))
model.add(Dense(3, activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['acc'])

history = model.fit(X_train, y_train, batch_size = 128, epochs = 15)

### 훈련된 모델로 예측, Submission 파일 생성

In [ ]:
y_pred = model.predict_classes(X_test)
sample_submission['category'] = y_pred
sample_submission.to_csv('/kaggle/working/submission.csv', encoding = 'utf-8', index = False)